In [12]:
import sys
sys.path.append("..")

import tensorflow as tf
import nltk
import random
import datetime

from data_encoders.text_encoder import EncodingProletariat

In [2]:
print(nltk.corpus.gutenberg.fileids())
emma = nltk.corpus.gutenberg.words('austen-emma.txt')

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
num_inputs = 3
batch_size = 100


In [6]:
ep = EncodingProletariat(emma, num_inputs=num_inputs)
ep.encodings_x

array([[3310, 6605, 3207],
       [6605, 3207, 1933],
       [3207, 1933, 3334],
       ...,
       [1777, 6762, 5911],
       [6762, 5911,  798],
       [5911,  798, 4193]])

In [7]:
num_classes = len(ep.vocab_dict)

In [19]:
inputs = tf.placeholder(tf.int32, [batch_size, num_inputs], name="input")
targets = tf.placeholder(tf.float16, [batch_size, num_classes], name="target")

In [66]:
def embed(inputs, batch_size, num_inputs, embed_dim, vocab_size):
    with tf.name_scope("word_embeddings"):
        word_vectors = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1, 1, seed=datetime.datetime.now().microsecond))
        embeddings = tf.nn.embedding_lookup(word_vectors, inputs)
        
    return embeddings

In [70]:
embed(ep.encodings_x[0:100], batch_size=batch_size, num_inputs=num_inputs, embed_dim=10, vocab_size=num_classes)

<tf.Tensor 'word_embeddings/embedding_lookup/Identity:0' shape=(100, 3, 10) dtype=float32>